# Resumen
Se llevó a cabo un estudio utilizando datos de pacientes con COVID-19 en México, donde se desarrolló un modelo de Regresión Logística con el objetivo de predecir si un paciente requiere hospitalización.

Dentro del análisis, se aplicaron diversas técnicas para limpiar y asegurar la calidad de los datos, incluyendo la identificación y tratamiento de valores atípicos, así como la imputación de datos faltantes. Además, se implementaron estrategias para mitigar el subajuste y el desbalance de datos.

Los resultados mostraron que el modelo de Regresión Logística logró una precisión del 97.69% y una exactitud (accuracy) del 84%, lo que sugiere una solución efectiva para el problema estudiado

In [ ]:
from google.colab import drive

drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
import pickle
import pandas as pd
import numpy as np

import cufflinks as cf
import matplotlib.pyplot as plt
import missingno as msno
from plotly.offline import plot, iplot
import plotly.graph_objects as go
import plotly.express as px
#import chart_studio.plotly as py
cf.go_offline()

from  functools import reduce
from scipy import stats
from scipy.stats import shapiro
from scipy.stats import zscore
from scipy.stats import chisquare
from scipy.stats import ksone
import statsmodels.api as sm

import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PowerTransformer
from sklearn.impute import SimpleImputer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, roc_auc_score
from sklearn.model_selection import  KFold, RepeatedKFold, cross_val_score
from imblearn.under_sampling import RandomUnderSampler

import warnings
warnings.filterwarnings("ignore")

In [ ]:
## Funciones

def rename_cols(df,cols,prefix):
    new_feats=[prefix+col for col in cols]
    df=df.rename(columns=dict(zip(cols,new_feats)))
    return df

def unitaria(df,col):
    result = pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result.values[0] > .91) :
            print(f"{col} -- VARIABLE UNITARIA")
        result = result.map(lambda x : str(round(x*100,2))+"%")
        result.reset_index(inplace=True)
        result.columns = [col+"_valores","%_aparicion"]
    return result

def unitarias(df,col):
    result=pd.DataFrame(df[col].value_counts(1))
    if result.shape[0]>0:
        if (result[col].values[0]>.91) :
            print(f"{col} -- VARIABLE UNITARIA")

def completitud(df):
    comp=pd.DataFrame(df.isnull().sum())
    comp.reset_index(inplace=True)
    comp=comp.rename(columns={"index":"columna",0:"total"})
    comp["completitud"]=(1-comp["total"]/df.shape[0])*100
    comp=comp.sort_values(by="completitud",ascending=True)
    comp.reset_index(drop=True,inplace=True)
    return comp

def box(df,col,title):
    layout = go.Layout(font_family="Verdana, monospace",
                       font_color="black",
                       title_text=title,
                       title_font_size=20,
                        xaxis= {"title": {"font": {"family": 'Verdana, monospace',"size": 18,"color": '#002e4d'}}},
                       title_font_family="Georgia, monospace",
                       title_font_color="#1C5166",
                       template="plotly_white")
    fig=df[[col]].iplot(kind='box',title=title,asFigure=True,theme="white",layout=layout,color="#3190B5", boxpoints='outliers')
    fig.update_traces(opacity=0.90)
    return fig

def pie(df,col,title,x_title="",y_title=""):
    layout = go.Layout(template="plotly_white")
    colors=[ "#152337", "#183152","#17416d","#005096","#00569c","#005ba3","#0061a9","#1567af","#226cb6","#2c72bc", "#0061a9","#4c79b7","#7492c6","#98acd4","#bbc7e2","#dde3f1","#ffffff"
]
    aux=pd.DataFrame(df[col].value_counts()).reset_index().rename(columns={"count":"conteo"})
    fig=aux.iplot(kind='pie',labels=col,values="conteo",title=title,asFigure=True,theme="white")

    fig.update_traces()
    fig.show()

def eval_metrics(y_test, model_pred):
    prec = precision_score(y_test,model_pred)
    acc = accuracy_score(y_test,model_pred)
    recall = recall_score(y_test,model_pred)
    roc = roc_auc_score(y_test,model_pred)

    data ={'precision':[prec],
                'accuracy':[acc],
                'recall': [recall],
                'roc':[roc]}
    return pd.DataFrame(data)

# Calidad de datos

## Preview

In [ ]:
df = pd.read_pickle('/content/gdrive/MyDrive/data/COVID19MEXICO.pkl')

In [ ]:
df

In [ ]:
df.shape

In [ ]:
df.columns

In [ ]:
df.info()

In [ ]:
df.describe(include = "object")

In [ ]:
df.describe()

In [ ]:
for col in df:
  print(col)
  values = df[col].value_counts()
  display(values)
  print("\n")

In [ ]:
df.columns

## Etiquetado de variables

Borramos las variables "FECHA_DEF", "FECHA_INGRESO", "INTUBADO", "UCI" para seguir lineamientos de la práctica. Etiquetamos variables.

In [ ]:
df.drop(columns = ["FECHA_DEF", "FECHA_INGRESO", "INTUBADO", "UCI"], inplace = True)

In [ ]:
id_feats = ['ID_REGISTRO']
v_feats = ['ORIGEN', 'SECTOR', 'ENTIDAD_UM',
       'SEXO', 'ENTIDAD_NAC', 'ENTIDAD_RES', 'MUNICIPIO_RES',
       'NEUMONIA','NACIONALIDAD', 'EMBARAZO', 'HABLA_LENGUA_INDIG', 'INDIGENA',
       'DIABETES', 'EPOC', 'ASMA', 'INMUSUPR', 'HIPERTENSION', 'OTRA_COM',
       'CARDIOVASCULAR', 'OBESIDAD', 'RENAL_CRONICA', 'TABAQUISMO',
       'OTRO_CASO', 'TOMA_MUESTRA_LAB', 'RESULTADO_LAB',
       'TOMA_MUESTRA_ANTIGENO', 'RESULTADO_ANTIGENO', 'CLASIFICACION_FINAL',
       'MIGRANTE', 'PAIS_NACIONALIDAD', 'PAIS_ORIGEN']
c_feats = ['EDAD']

d_feats = ['FECHA_SINTOMAS']

tgt_feats = ['TIPO_PACIENTE']

df=rename_cols(df,id_feats,"id_")
df=rename_cols(df,d_feats,"d_")
df=rename_cols(df,v_feats,"v_")
df=rename_cols(df,c_feats,"c_")
df=rename_cols(df,tgt_feats,"tgt_")

## Duplicados

No se encontraron duplicados

In [ ]:
df.duplicated().sum()

In [ ]:
df.duplicated(subset = 'id_ID_REGISTRO').sum()

## Orden y precisión

Buscamos y eliminamos variables unitarias. Normalizamos aquellas que tengan demasiadas categorías.

In [ ]:
for col in df.filter(like="v_").columns:
    unitarias(df,col)

In [ ]:
df.drop(columns = ["v_PAIS_ORIGEN", "v_PAIS_NACIONALIDAD", "v_MIGRANTE", "v_RENAL_CRONICA", "v_CARDIOVASCULAR",
                   "v_OTRA_COM","v_NEUMONIA", "v_NACIONALIDAD","v_HABLA_LENGUA_INDIG", "v_INDIGENA",
                   "FECHA_ACTUALIZACION", ## unitarias
                   "v_MUNICIPIO_RES"] ## con demasiadas categorías, usaremos únicamente el estado de residencia
        , inplace = True)

Mantenemos las variables de TABAQUISMO y ASMA, ya que se consideran importantes para predecir la hospitalización de los pacientes de COVID.

Normalizamos variables de *estado de residencia* y *nacimiento* así como la *entidad de la unidad médica*, el *sector* y la *clasificación final*, por tener muchas categorías. Las variables relacionadas a entidades se convierten en variables binarias:

- Zona metropolitana: Estado de México y CDMX
- Otros: Estados diferentes de la zona metropolitana.

Para SECTOR, las 10 menos frecuentes se agrupan en "otros". Finalmente, para CLASIFICACION FINAL se agrupan las 5 menos frecuentes en una categoría "otros".

In [ ]:
otrosNAC = list(set(df["v_ENTIDAD_NAC"].unique())-set([9,15]))
df["v_ENTIDAD_NAC"].replace([9, 15], "zona_metropolitana", inplace = True)
df["v_ENTIDAD_NAC"].replace(otrosNAC, "otros", inplace = True)

In [ ]:
otrosRES = list(set(df["v_ENTIDAD_RES"].unique())-set([9,15]))
df["v_ENTIDAD_RES"].replace([9, 15], "zona_metropolitana", inplace = True)
df["v_ENTIDAD_RES"].replace(otrosRES, "otros", inplace = True)

In [ ]:
otrosUM = list(set(df["v_ENTIDAD_UM"].unique())-set([9,15]))
df["v_ENTIDAD_UM"].replace([9, 15], "zona_metropolitana", inplace = True)
df["v_ENTIDAD_UM"].replace(otrosUM, "otros", inplace = True)

In [ ]:
dict_sector = dict(zip(list(df["v_SECTOR"].value_counts(1).iloc[-10:].index),["otro"]*10))
df["v_SECTOR"] = df["v_SECTOR"].replace(dict_sector)

In [ ]:
dict_clas_final = dict(zip(list(df["v_CLASIFICACION_FINAL"].value_counts(1).iloc[-5:].index),["otro"]*5))
df["v_CLASIFICACION_FINAL"] = df["v_CLASIFICACION_FINAL"].replace(dict_clas_final)

## Valores faltantes

Los datos faltantes en las variables categóricas son representados por los códigos 99, 97 y 98, por lo que las observaciones con dichos valores serán convertidas a NA.

Se eliminan del dataset las variables con menos del 80% de los datos completos.

In [ ]:
df_missings = df.drop("c_EDAD",axis=1)

In [ ]:
df_missings.replace(99, np.nan, inplace = True)
df_missings.replace(97, np.nan, inplace = True)
df_missings.replace(98, np.nan, inplace = True)

In [ ]:
df_missings['c_EDAD'] = df["c_EDAD"]

In [ ]:
completitud(df_missings)

In [ ]:
df = df_missings.copy()
df.drop(columns = ["v_RESULTADO_LAB", "v_EMBARAZO"], inplace = True)

Para hacer el EDA tomamos solo una muestra del conjunto de datos, debido al tamaño de éste.

In [ ]:
X, X_dummie =train_test_split(df,test_size=0.9,stratify=df["tgt_TIPO_PACIENTE"],random_state=54)

## EDA

In [ ]:
for col in X.filter(like="v_"):
    pie(X,col,col)

In [ ]:
colors = sns.dark_palette("#69d", reverse=True)

antigeno=pd.crosstab(df.v_RESULTADO_ANTIGENO,df.tgt_TIPO_PACIENTE,normalize=True)
antigeno

In [ ]:
antigeno.plot(kind="bar",stacked=True,color=colors)

In [ ]:
sns.boxplot( data=X,x='tgt_TIPO_PACIENTE', y='c_EDAD');

## Valores atípicos

Se eliminan las edades mayores de 105 años, ya que o son datos incorrectos o muy extraños, además de que representan menos del 0.1% de los datos.

In [ ]:
plt.scatter(range(df.shape[0]), y=df['c_EDAD'])

In [ ]:
df[df["c_EDAD"]>105].shape[0]/df.shape[0]

In [ ]:
df.drop(df[df["c_EDAD"]>105].index, inplace=True, axis=0)

## Split train-test
Partimos la muestra en train y test para realizar la imputación de los valores faltantes.

In [ ]:
y = df['tgt_TIPO_PACIENTE']
X = df.drop('tgt_TIPO_PACIENTE',axis=1)
y.replace(2, 0, inplace=True)

X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    random_state=1,
                                                    stratify=df['tgt_TIPO_PACIENTE'],
                                                    test_size=0.25,
                                                    shuffle = True)

In [ ]:
print(X_train.shape)
print(X_test.shape)

(853416, 21)
(284472, 21)


In [ ]:
y_train.value_counts(1)

1    0.922576
0    0.077424
Name: tgt_TIPO_PACIENTE, dtype: float64

## Missings

Se imputan todas las columnas con algún valor faltante y esto se hace con la moda.

In [ ]:
imp1=SimpleImputer(missing_values=np.nan,strategy="most_frequent")

In [ ]:
miss=completitud(df)
miss[miss["completitud"]<100]
comple = miss[miss["completitud"]<100].columna
comple

0    v_RESULTADO_ANTIGENO
1             v_OTRO_CASO
2              v_DIABETES
3            v_TABAQUISMO
4                  v_EPOC
5          v_HIPERTENSION
6                  v_ASMA
7              v_INMUSUPR
8              v_OBESIDAD
Name: columna, dtype: object

In [ ]:
for col in comple:
    imp1.fit(X_train[[col]])
    X_train[[col]]=imp1.transform(X_train[[col]])
    X_test[[col]]=imp1.transform(X_test[[col]])

Hasta aquí los datos faltantes han sido imputados.

In [ ]:
# resultados imputacion: vistazo rápido
for col in X_train.filter(like="v_").columns:
    print(X_train[col].value_counts())
    print("\n")

2    603457
1    249959
Name: v_ORIGEN, dtype: int64


4       458303
12      268351
9        65149
6        42332
otro     19281
Name: v_SECTOR, dtype: int64


otros                 550072
zona_metropolitana    303344
Name: v_ENTIDAD_UM, dtype: int64


1    501720
2    351696
Name: v_SEXO, dtype: int64


otros                 551103
zona_metropolitana    302313
Name: v_ENTIDAD_NAC, dtype: int64


otros                 554056
zona_metropolitana    299360
Name: v_ENTIDAD_RES, dtype: int64


2.0    786049
1.0     67367
Name: v_DIABETES, dtype: int64


2.0    845041
1.0      8375
Name: v_EPOC, dtype: int64


2.0    833426
1.0     19990
Name: v_ASMA, dtype: int64


2.0    845707
1.0      7709
Name: v_INMUSUPR, dtype: int64


2.0    759665
1.0     93751
Name: v_HIPERTENSION, dtype: int64


2.0    788075
1.0     65341
Name: v_OBESIDAD, dtype: int64


2.0    818606
1.0     34810
Name: v_TABAQUISMO, dtype: int64


2.0    738496
1.0    114920
Name: v_OTRO_CASO, dtype: int64


2    696227
1    1

Volvemos a contar datos faltantes

In [ ]:
X_train.isna().sum()

## One hot encoding

Codificamos las variables categóricas. Se eliminan las variables de ID y la fecha de síntomas (FECHA_SINTOMAS) por no considerarse relevantes para el modelo.

In [ ]:
X_train.columns

## Preprocesamiento de datos

Se aplica One hot encoding a las variables categóricas y escalamiento a la continua.

In [ ]:
var_cat = X_train.filter(like = "v_").columns
preprocesado = ColumnTransformer(
    [('scale', StandardScaler(), ['c_EDAD']), #escalamos edad
    ('one', OneHotEncoder(sparse=False,drop='first'),var_cat)], #onehot encoding
    remainder='drop'
)

#convertimos categoricas a object
for col in X_train.filter(like="v_").columns:
    X_train[col]=X_train[col].astype('str')
    X_test[col]=X_test[col].astype('str')

In [ ]:
X_train_pp = preprocesado.fit_transform(X_train,y_train)
X_test_pp = preprocesado.transform(X_test)

X_train_pp = pd.DataFrame(columns=preprocesado.get_feature_names_out(), data=X_train_pp)
X_test_pp = pd.DataFrame(columns=preprocesado.get_feature_names_out(), data=X_test_pp)

In [ ]:
X_train_pp.shape

(853416, 23)

In [ ]:
X_train_pp

,scale__c_EDAD,one__v_ORIGEN_2,one__v_SECTOR_4,one__v_SECTOR_6,one__v_SECTOR_9,one__v_SECTOR_otro,one__v_ENTIDAD_UM_zona_metropolitana,one__v_SEXO_2,one__v_ENTIDAD_NAC_zona_metropolitana,one__v_ENTIDAD_RES_zona_metropolitana,...,one__v_INMUSUPR_2.0,one__v_HIPERTENSION_2.0,one__v_OBESIDAD_2.0,one__v_TABAQUISMO_2.0,one__v_OTRO_CASO_2.0,one__v_TOMA_MUESTRA_LAB_2,one__v_TOMA_MUESTRA_ANTIGENO_2,one__v_RESULTADO_ANTIGENO_2.0,one__v_CLASIFICACION_FINAL_7,one__v_CLASIFICACION_FINAL_otro
0,-0.018787,0.0,0.0,0.0,0.0,0.0,1.0,1.0,1.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
1,0.032650,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
2,1.267131,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
3,2.141556,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
4,0.084086,1.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
853411,0.238397,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,...,1.0,1.0,0.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0
853412,0.186960,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,0.0,0.0,1.0,1.0,0.0
853413,-0.533154,1.0,0.0,1.0,0.0,0.0,1.0,0.0,0.0,1.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,1.0,1.0,0.0
853414,-0.430281,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,1.0,1.0,1.0,1.0,1.0,1.0,0.0,0.0,0.0,0.0


In [ ]:
X_train.reset_index(inplace=True, drop=True)
X_test.reset_index(inplace=True, drop=True)
y_train.reset_index(inplace=True, drop=True)
y_test.reset_index(inplace=True, drop=True)

# Regresión Logística

## Primer modelo
Se utilizan todas las variables y vemos coeficientes y  performance.

In [ ]:
X_train_const = sm.add_constant(X_train_pp)

logit_model = sm.Logit(y_train, X_train_const)

logit_result = logit_model.fit()
std_errors = logit_result.bse

In [ ]:
logit_result.summary()

In [ ]:
model = LogisticRegression(penalty='none', fit_intercept=True).fit(X = X_train_pp, y = y_train)

#esto lo usamos para ver que los coeficientes fueran los mismos y sí
#coeficientes_df = pd.DataFrame({'Variable': X_train_pp.columns, 'Coeficiente': model.coef_.flatten()})

In [ ]:
# Evaluacion - matriz de confusion
display(pd.DataFrame(confusion_matrix(y_train,model.predict(X_train_pp))))

In [ ]:
model_prediction = model.predict(X_train_pp)

eval_metrics(y_train,model_prediction)

## Segundo modelo
Quitando 'one__v_ENTIDAD_NAC_zona_metropolitana', 'one__v_TABAQUISMO_2.0', ya que en el anterior modelo no son significativas de acuerdo al P-value obtenido (mayor a 0.05)

In [ ]:
X_train_m2 = X_train_pp.drop(['one__v_ENTIDAD_NAC_zona_metropolitana', 'one__v_TABAQUISMO_2.0'], axis=1)

In [ ]:
model2 = LogisticRegression(penalty='none', fit_intercept=True).fit(X = X_train_m2, y = y_train)

### Train

Evaluamos el conjunto train

In [ ]:
model_prediction2 = model2.predict(X_train_m2)
display(pd.DataFrame(confusion_matrix(y_train,model_prediction2)))

In [ ]:
eval_metrics(y_train,model_prediction2)

Calculamos la precisión con Cross Validation para el conjunto train. Se elige la precisión como la métrica que nos interesa en este procedimiento, debido a que se tiene un dataset desbalanceado.

In [ ]:
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
scores_rkf = cross_val_score(model2, X_train_m2, y_train, cv=rkf, scoring='precision')

print(f"Scores con RepeatedKFold: {scores_rkf}")
print(f"Promedio de Scores con RepeatedKFold: {np.mean(scores_rkf)}")
print(f"Desviación estándar de Scores con RepeatedKFold: {np.std(scores_rkf)}")

### Test
Evaluamos para el conjunto Test.Se elige la precisión como la métrica que nos interesa en este procedimiento, debido a que se tiene un dataset desbalanceado.

Como resultado se obtiene que las métricas son ligeramente mejores que para el Train (por algunas centésimas para todas las métricas).

In [ ]:
X_test_m2 = X_test_pp.drop(['one__v_ENTIDAD_NAC_zona_metropolitana', 'one__v_TABAQUISMO_2.0'], axis=1)

In [ ]:
model_prediction2_test = model2.predict(X_test_m2)

# Matriz de confusion
display(pd.DataFrame(confusion_matrix(y_test,model_prediction2_test)))

In [ ]:
eval_metrics(y_test,model_prediction2_test)

In [ ]:
rkf = RepeatedKFold(n_splits=5, n_repeats=3, random_state=42)
scores_rkf = cross_val_score(model2, X_test_m2, y_test, cv=rkf, scoring='precision')

print(f"Scores con RepeatedKFold: {scores_rkf}")
print(f"Promedio de Scores con RepeatedKFold: {np.mean(scores_rkf)}")
print(f"Desviación estándar de Scores con RepeatedKFold: {np.std(scores_rkf)}")

## Undersampling

Debido al posible underfitting de arriba, se entrena un modelo de regresión logística con todos los datos donde las personas fueron hospitalizadas y 80,000 donde no lo fueron, ya que el modelo está recibiendo muy poca información de los primeros (pues se tiene un desbalance notable). Se eligen más observaciones de la clase "no hospitalizados", para mantener que la mayoría de los datos están en esa clase.

El modelo generado se evalúa en los mismos conjuntos train y test de arriba. Se obtiene un mejor performance y se elimina el underfitting para las evaluaciones en dichos conjuntos, pero si se realiza el una Validación Cruzada, el performance se mantiene y persiste el pequeño underfitting.

In [ ]:
under = RandomUnderSampler(random_state=0, sampling_strategy={0:66075 , 1: 80000})

X_res, y_res = under.fit_resample(X_train_m2, y_train)

In [ ]:
y_res.value_counts(1)

1    0.547664
0    0.452336
Name: tgt_TIPO_PACIENTE, dtype: float64

In [ ]:
model_s = LogisticRegression(penalty='none', fit_intercept=True).fit(X = X_res, y = y_res)

### Train

In [ ]:
model_prediction_s = model_s.predict(X_train_m2)
display(pd.DataFrame(confusion_matrix(y_train,model_prediction_s)))

In [ ]:
eval_metrics(y_train,model_prediction_s)

Cross Validation

Se elige la precisión como la métrica que nos interesa en este procedimiento, debido a que se tiene un dataset desbalanceado.

In [ ]:
rkf3 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=111)
scores_rkf3 = cross_val_score(model_s, X_train_m2, y_train, cv=rkf3, scoring='precision')

print(f"Scores con RepeatedKFold: {scores_rkf3}")
print(f"Promedio de Scores con RepeatedKFold: {np.mean(scores_rkf3)}")
print(f"Desviación estándar de Scores con RepeatedKFold: {np.std(scores_rkf3)}")

### Test

In [ ]:
model_s_prediction_test = model_s.predict(X_test_m2)

# Matriz de confusion
display(pd.DataFrame(confusion_matrix(y_test,model_s_prediction_test)))

In [ ]:
eval_metrics(y_test, model_pred=model_s_prediction_test)

Cross Validation

Se elige la precisión como la métrica que nos interesa en este procedimiento, debido a que se tiene un dataset desbalanceado.

In [ ]:
rkf4 = RepeatedKFold(n_splits=5, n_repeats=3, random_state=111)
scores_rkf4 = cross_val_score(model_s, X_test_m2, y_test, cv=rkf4, scoring='precision')

print(f"Scores con RepeatedKFold: {scores_rkf4}")
print(f"Promedio de Scores con RepeatedKFold: {np.mean(scores_rkf4)}")
print(f"Desviación estándar de Scores con RepeatedKFold: {np.std(scores_rkf4)}")

In [ ]:
pd.to_pickle((preprocesado2,lr3),'./01_Models/model_lr3.pkl')

filename = 'regression_logistica_model.pkl'
pickle.dump(model1_lars, open(filename, 'wb'))